In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tableone import TableOne
import os

# To Show All Columns in DataFrame or numpy arrays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)

# To Get Rid of Warnings
import warnings
warnings.filterwarnings("ignore")

In [24]:
# PatAddressDim

pataddressdim = pd.read_csv('/home/idies/workspace/Storage/ccai6/NLP in DR/Geocoding/Input/3_16_ratio_status.csv', encoding='latin-1')

pataddressdimcol ={'e_mrn': "e_mrn",
 'adi_state': "origAdd",
 'address': "newAdd",
 'M_STATE': "stateAbbrev",
 'county': "county2",
 'CountyFp': "county",                   
 'Tract': "tract",
 'BlkGrp': "block"
 }

pataddressdim = pataddressdim.rename(columns=pataddressdimcol)

#pataddressdim['status']=pataddressdim['status'].str.strip()
pataddressdim = pataddressdim.loc[:, ~pataddressdim.columns.isin(['address_start', 'address_end', 'zip4', 'longitude', 'latitude', 'adi_nat'])]

In [43]:
def stateNum(name) -> str:
    switcher = {
        "AK": "02",
        "AL": "01",
        "AR": "05",
        "AS": "60",
        "AZ": "04",
        "CA": "06",
        "CO": "08",
        "CT": "09",
        "DC": "11",
        "DE": "10",
        "FL": "12",
        "GA": "13",
        "GU": "66",
        "HI": "15",
        "IA": "19",
        "ID": "16",
        "IL": "17",
        "IN": "18",
        "KS": "20",
        "KY": "21",
        "LA": "22",
        "MA": "25",
        "MD": "24",
        "ME": "23",
        "MI": "26",
        "MN": "27",
        "MO": "29",
        "MS": "28",
        "MT": "30",
        "NC": "37",
        "ND": "38",
        "NE": "31",
        "NH": "33",
        "NJ": "34",
        "NM": "35",
        "NV": "32",
        "NY": "36",
        "OH": "39",
        "OK": "40",
        "OR": "41",
        "PA": "42",
        "PR": "72",
        "RI": "44",
        "SC": "45",
        "SD": "46",
        "TN": "47",
        "TX": "48",
        "UT": "49",
        "VA": "51",
        "VI": "78",
        "VT": "50",
        "WA": "53",
        "WI": "55",
        "WV": "54",
        "WY": "56",
    }
    num = switcher.get(name)
    if num:
        return num
    else:
        return None 

def countyNum(name): #add preceding zeros to county code
    while len(name) < 3:
        name = "0" + name
    return name

def tractNum(name): #add preceding zeros to tract code
    while len(name) < 6:
        name = "0" + name
    return name

def convert(name): #return string version of zipcode 
    if not pd.isna(name):
        return str(int(name))
    else:
        return ""

In [53]:
df = pataddressdim

mrn = []
original = []
new = []
fips = []
oFips = []
match = []
matchList = []
state = []
county = []
tract = []
block = []
less = 0

for i in range(df.shape[0]):
    fMatch = False
    if i % 10000 == 0:
        print(i)
    currMrn = df.iloc[i]['e_mrn']
    oAdd = str(df.iloc[i]['origAdd']) + " " + str(df.iloc[i]['city']) + ", " + str(df.iloc[i]['state']) + " " + convert(df.iloc[i]['zipcode'])
    nAdd = str(df.iloc[i]['newAdd']) + " " + str(df.iloc[i]['city']) + ", " + str(df.iloc[i]['state']) + " " + convert(df.iloc[i]['zipcode'])
    matchLevel = df.iloc[i]['_MATCHED_']
    oMatchLevel = df.iloc[i]['status']
    if not pd.isna(df.iloc[i]['county']) and not pd.isna(df.iloc[i]['tract']) and not pd.isna(df.iloc[i]['block']):
        currState = stateNum(str(df.iloc[i]['stateAbbrev']))
        currCounty = countyNum(str(int(df.iloc[i]['county'])))  
        currTract = tractNum(str(int(df.iloc[i]['tract'])))
        currBlock = str(int(df.iloc[i]['block']))
        if not currState:
            fip = "?"
        else:
            fip = currState + currCounty + currTract + currBlock
    else:
        fip = "?"
    if not pd.isna(df.iloc[i]['census']) and not pd.isna(df.iloc[i]['_15']):
        oFip1 = str(int(df.iloc[i]['census']))                    
        oFip2 = tractNum(str(int(df.iloc[i]['_15'])))
        if len(fip) == 11 or fip[0] == "0":
            less = less + 1
            oFip2 = oFip2[:-1]
        oFip = oFip1 + oFip2
    else:
        oFip = "?"
    if len(fip) == 11:
        fip = "0" + fip
    if len(oFip) == 11:
        oFip = "0" + oFip
    if fip.strip == oFip.strip():
        fMatch = True
        state.append(True)
        county.append(True)
        tract.append(True)
        block.append(True)
    else:
        if matchLevel == "Street" and oMatchLevel == "Street":
            fMatch = "street"
            nState = fip[0:2]
            oState = oFip[0:2]
            nCounty = fip[2:5]
            oCounty = oFip[2:5]
            nTract = fip[5:11]
            oTract = oFip[5:11]
            nBlock = fip[11:12]
            oBlock = oFip[11:12]            
            if nState == oState:
                state.append(True)
            else:
                state.append(False)
            if nCounty == oCounty:
                county.append(True)
            else:
                county.append(False)
            if nTract == oTract:
                tract.append(True)
            else:
                tract.append(False)
            if nBlock == oBlock:
                block.append(True)
            else:
                block.append(False)
        else:
            fMatch = False
            state.append(True)
            county.append(True)
            tract.append(True)
            block.append(True)
    mrn.append(currMrn)
    original.append(oAdd)
    new.append(nAdd)
    fips.append(fip)
    oFips.append(oFip)
    match.append(fMatch)
    matchList.append(matchLevel)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
170


In [55]:
newDf = pd.DataFrame(list(zip(mrn, original, new, fips, oFips, match, matchList, state, county, tract, block)), columns =['MRN', 'original', 'new', 'fips', 'oFips', 'match', 'matchList', 'state', 'county', 'tract', 'block'])
newDf.to_csv('/home/idies/workspace/Storage/ccai6/NLP in DR/Geocoding/Output/fips_Street_Sep_3_21_3.csv')#includes indexes